Notebook used for the compuation of the SL_4 chain complex. The SL_5 chain complex required optimising the code, so this is outdated. 

In [1]:
using Pkg
Pkg.activate(normpath(joinpath(@__DIR__, "../../")))

  Activating project at `~/HigherTSL3/SLnCohomology`


In [2]:
Pkg.add("Serialization")

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/HigherTSL3/SLnCohomology/Project.toml`
  No Changes to `~/HigherTSL3/SLnCohomology/Manifest.toml`


Precompiling project...


  ✓ SLnCohomology


  1 dependency successfully precompiled in 10 seconds. 140 already precompiled.


In [3]:
using LinearAlgebra
using Combinatorics
using Serialization

# Functions
Should probably be moved to a separate file at some point.

In [4]:
function signed_matrices(dimension)
    # Returns all diagonal square "sign matrices" in dimension
    signs_list = []
    for set in powerset(1:dimension)
        matrix = zeros(Int8,dimension,dimension)
        for i in 1:dimension
            if i in set
                matrix[(i-1)*dimension+(i)] = -1
            else
                matrix[(i-1)*dimension+(i)] = 1
            end
        end
        push!(signs_list,matrix)
    end    
    return signs_list
end

signed_matrices (generic function with 1 method)

In [5]:
function bases_in_integer_lattices(matrix)
    #= matrix an integer matrix whose columns form a basis of the lattice
    returns all sets of bases, with all possible sign combinations 
    =#
    bases = []
    columns = collect(eachcol(matrix)) # set of all columns
    dimension = length(columns[1])
    sign_changes = signed_matrices(dimension)
    for candidate in Combinatorics.permutations(columns,dimension) # all subsets of columns of with dimension-many elements
        matrix_candidate = transpose(reduce(vcat,transpose.(candidate)))
        if (round(det(matrix_candidate)) == 1) || (round(det(matrix_candidate)) == -1)
            #if that's the case, candidate is a basis of Z^dimension
            for sign in sign_changes
                # each basis element can be multiplied by +-1. Add all combinations
                push!(bases, matrix_candidate*sign)
            end
        end
    end
    return bases
end

bases_in_integer_lattices (generic function with 1 method)

In [6]:
function relative_orientation(matrix1,matrix2)
    #= matrix1, matrix2 need to be integer matrices such that the set of lines spanned by the columns of 
        matrix1 is the same as that of matrix2
        computes the orientation of the simplex spanned by the columns
    =# 
    permutation = Int64[]
    for vector in eachcol(matrix2)
        vertex_index_in_matrix1 = 1
        vector_is_column_of_matrix1 = false
        for vector_standard in eachcol(matrix1)
            if vector == vector_standard || -vector == vector_standard
                push!(permutation,vertex_index_in_matrix1)
                vector_is_column_of_matrix1 = true
            end
            vertex_index_in_matrix1 += 1
        end
        vector_is_column_of_matrix1 || error("matrix1 isn't a permutation of matrix2.")
    end
    sign = Combinatorics.levicivita(permutation)
    return sign
end

relative_orientation (generic function with 1 method)

In [7]:
function quadratic_form(matrix)
    form = matrix*transpose(matrix)
    return form
end

quadratic_form (generic function with 1 method)

In [8]:
function stabiliser_coset_with_orientation(matrix1, matrix2)
    #= 
    matrix1, matrix2 integer matrix whose columns form a basis of the lattice.
    Returns all (g,orientation) such that g\in SL_dimension(Z) sends the quadratic form associated to matrix1 to 
    the one associated to matrix 2 and orientation is the relative orientation of the corresponding simplices
    =#
    elements = []
    dimension = size(matrix1)[1]
    quadratic_form1 = quadratic_form(matrix1) # the quadratic form associated to the lattice
    quadratic_form2 = quadratic_form(matrix2) # the quadratic form associated to the lattice
    bases_in_image = bases_in_integer_lattices(matrix2) # bases in the image
    basis_in_matrix_1 = bases_in_integer_lattices(matrix1)[1] # a fixed basis in the domain
    basis_in_matrix_1_inverse = Int.(basis_in_matrix_1^(-1))  #inverse as integer matrix
    for basis in bases_in_image
        g_transpose = basis*basis_in_matrix_1_inverse # this sends the fixed basis_in_matrix_1 to basis in matrix2
        if round(det(g_transpose)) == 1
            # then it lies in SL
            g = transpose(g_transpose)
            if (g_transpose*quadratic_form1*g == quadratic_form2) # then g sends the first form to the second one
                orientation = relative_orientation(g_transpose*matrix1,matrix2)
                # this is the relative orientation of g.first cell compared to second cell
                push!(elements, (g,orientation))
            end
        end
    end
    return elements
end

stabiliser_coset_with_orientation (generic function with 1 method)

In [9]:
function same_orbit(matrix1,matrix2)
    #= Checks whether matrix1 lies in the SL_n orbit as matrix2
    =#
    if length(stabiliser_coset_with_orientation(matrix1,matrix2)) > 0
        return true
    else
        return false
    end
end

same_orbit (generic function with 1 method)

In [10]:
function orbit_in_list(matrix,list)
    #= Checks whether matrix lies in the SL_n orbit of one of the elements in list
    =#
    for matrix2 in list
        if same_orbit(matrix,matrix2)
            return true
        end
    end
    return false
end

orbit_in_list (generic function with 1 method)

In [11]:
function cell_in_interior(matrix)
    #= check whether the matrix determines a cell in the interior of the symmetric space
    =#
    if length(bases_in_integer_lattices(matrix))>0
        return true
    else
        return false
    end
end

cell_in_interior (generic function with 1 method)

In [12]:
function boundaries_in_group_ring_with_orientation(matrix,cell_dictionary)
    #= cell dictionary should have the cells of the complex
    This only works if the cells are simplicial!
    =#
    boundary_cells = []
    cell_dimension = size(matrix)[2] - 1 # only true if the cell is simplicial
    for j in 1:size(matrix)[2]
        sign = (-1)^(j-1) # the sign of the differential
        boundary = matrix[begin:end,begin:end .!=j] # exclude the j-th column
        if cell_in_interior(boundary)
            boundary_information = Dict()
            boundary_information["sign"] = sign
            for cell_index in 1:length(cell_dictionary[cell_dimension-1])
                # check in the list of codimension 1 cells which orbit we have
                standard_cell = cell_dictionary[cell_dimension-1][cell_index]
                if same_orbit(standard_cell,boundary)
                    boundary_information["orbit_standard_cell"] = cell_index
                    boundary_information["orbit_coset_with_orientation"] = stabiliser_coset_with_orientation(standard_cell,boundary)
                end
            end
            push!(boundary_cells,boundary_information)
        end
    end
    return boundary_cells
end

boundaries_in_group_ring_with_orientation (generic function with 1 method)

# The SL_4 data

## GL_4 orbits in dimension >5
Originally got this data from a meeting with Dan Yasaki at BIRS.

In [13]:
# list of vertices
vertices_Voronoi_4 = [[ 0, 0, 0, 1 ],
[ 0, 0, 1, -1 ],
[ 0, 0, 1, 0 ],
[ 0, 1, -1, 0 ],
[ 0, 1, 0, -1 ],
[ 0, 1, 0, 0 ],
[ 0, 1, 1, -1 ],
[ 1, -1, 0, 0 ],
[ 1, 0, -1, 0 ],
[ 1, 0, 0, -1 ],
[ 1, 0, 0, 0 ],
[ 1, 0, 1, -1 ],
[ 1, 1, 0, -1 ]
]

13-element Vector{Vector{Int64}}:
 [0, 0, 0, 1]
 [0, 0, 1, -1]
 [0, 0, 1, 0]
 [0, 1, -1, 0]
 [0, 1, 0, -1]
 [0, 1, 0, 0]
 [0, 1, 1, -1]
 [1, -1, 0, 0]
 [1, 0, -1, 0]
 [1, 0, 0, -1]
 [1, 0, 0, 0]
 [1, 0, 1, -1]
 [1, 1, 0, -1]

Number of orbits of simplices of different types, starting from lowest dimension 3 up to highest dimension 9:

[1, 3, 4, 4, 2, 2, 2]

This only includes those cells that intersect the interior of the symmetric space non-trivially.

In [14]:
# types of 6-dimensional simplices for GL_4:
index6_1 = [3,5,6,7,8,9,10]
type6_1 = reduce(hcat,[vertices_Voronoi_4[index] for index in index6_1])
index6_2 = [3,5,6,8,9,10,11]
type6_2 = reduce(hcat,[vertices_Voronoi_4[index] for index in index6_2])
index6_3 = [3,5,6,8,10,11,13]
type6_3 = reduce(hcat,[vertices_Voronoi_4[index] for index in index6_3])
index6_4 = [3,5,6,8,9,11,13]
type6_4 = reduce(hcat,[vertices_Voronoi_4[index] for index in index6_4])

# types of 7-dimensional simplices for GL_4:
index7_1 = [3,5,6,7,8,9,10,11]
type7_1 = reduce(hcat,[vertices_Voronoi_4[index] for index in index7_1])
index7_2 = [3,5,6,8,9,10,11,13]
type7_2 = reduce(hcat,[vertices_Voronoi_4[index] for index in index7_2])

# types of 8-dimensional simplices for GL_4:
index8_1 = [3,5,6,7,8,9,10,11,13]
type8_1 = reduce(hcat,[vertices_Voronoi_4[index] for index in index8_1])
index8_2 = [3,6,7,8,9,10,11,12,13]
type8_2 = reduce(hcat,[vertices_Voronoi_4[index] for index in index8_2])

# 9-dimensional simplices missing for now (one of them non-simplicial)

cells_GL4 = Dict()
cells_GL4[6] = [type6_1,type6_2,type6_3,type6_4]
cells_GL4[7] = [type7_1,type7_2]
cells_GL4[8] = [type8_1,type8_2]

2-element Vector{Matrix{Int64}}:
 [0 0 … 1 1; 0 1 … 0 1; 1 0 … 0 0; 0 -1 … 0 -1]
 [0 0 … 1 1; 0 1 … 0 1; 1 0 … 1 0; 0 0 … -1 -1]

In [15]:
# Actually, the forms here also seem to be determined by the their determinant.
# However, I don't use this below (reduce speed but avoid potential problems)
for (dimension,cells) in cells_GL4
    for cell in cells
        println(det(quadratic_form(cell)))
    end
end

23.99999999999999
20.999999999999996
16.0
19.999999999999996
45.00000000000001
40.0
74.99999999999997
80.99999999999999


## Add SL orbits for dimension >5

In [16]:
cells_SL4 = Dict()
cells_SL4[6] = [type6_1,type6_2,type6_3,type6_4]
cells_SL4[7] = [type7_1,type7_2]
cells_SL4[8] = [type8_1,type8_2]

2-element Vector{Matrix{Int64}}:
 [0 0 … 1 1; 0 1 … 0 1; 1 0 … 0 0; 0 -1 … 0 -1]
 [0 0 … 1 1; 0 1 … 0 1; 1 0 … 1 0; 0 0 … -1 -1]

In [17]:
flip_matrix = [-1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1]
# multiplying with this matrix gives the potential second SL_n orbit contained in a GL_n orbit

4×4 Matrix{Int64}:
 -1  0  0  0
  0  1  0  0
  0  0  1  0
  0  0  0  1

In [18]:
for (dimension,cells) in cells_GL4
    for cell in cells
        SL_orbit = flip_matrix*cell
        # check whether its in the orbit of one of the other cells
        if ! orbit_in_list(SL_orbit,cells_SL4[dimension])
            push!(cells_SL4[dimension],SL_orbit)
        end         
    end
end

In [19]:
cells_SL4

Dict{Any, Any} with 3 entries:
  6 => [[0 0 … 1 1; 0 1 … 0 0; 1 0 … -1 0; 0 -1 … 0 -1], [0 0 … 1 1; 0 1 … 0 0;…
  7 => [[0 0 … 1 1; 0 1 … 0 0; 1 0 … 0 0; 0 -1 … -1 0], [0 0 … 1 1; 0 1 … 0 1; …
  8 => [[0 0 … 1 1; 0 1 … 0 1; 1 0 … 0 0; 0 -1 … 0 -1], [0 0 … 1 1; 0 1 … 0 1; …

In [20]:
cells_SL4[6]

4-element Vector{Matrix{Int64}}:
 [0 0 … 1 1; 0 1 … 0 0; 1 0 … -1 0; 0 -1 … 0 -1]
 [0 0 … 1 1; 0 1 … 0 0; 1 0 … 0 0; 0 -1 … -1 0]
 [0 0 … 1 1; 0 1 … 0 1; 1 0 … 0 0; 0 -1 … 0 -1]
 [0 0 … 1 1; 0 1 … 0 1; 1 0 … 0 0; 0 -1 … 0 -1]

In [21]:
cells_SL4[7]

2-element Vector{Matrix{Int64}}:
 [0 0 … 1 1; 0 1 … 0 0; 1 0 … 0 0; 0 -1 … -1 0]
 [0 0 … 1 1; 0 1 … 0 1; 1 0 … 0 0; 0 -1 … 0 -1]

In [22]:
cells_SL4[8]

2-element Vector{Matrix{Int64}}:
 [0 0 … 1 1; 0 1 … 0 1; 1 0 … 0 0; 0 -1 … 0 -1]
 [0 0 … 1 1; 0 1 … 0 1; 1 0 … 1 0; 0 0 … -1 -1]

Maye a bit surprising: There seem to be no new SL_n orbits.

### Some sanity checks here

In [23]:
unflipped_matrix = cells_SL4[6][1]

4×7 Matrix{Int64}:
 0   0  0   0   1   1   1
 0   1  1   1  -1   0   0
 1   0  0   1   0  -1   0
 0  -1  0  -1   0   0  -1

In [24]:
flipped_matrix = flip_matrix*unflipped_matrix

4×7 Matrix{Int64}:
 0   0  0   0  -1  -1  -1
 0   1  1   1  -1   0   0
 1   0  0   1   0  -1   0
 0  -1  0  -1   0   0  -1

In [25]:
g = stabiliser_coset_with_orientation(flipped_matrix, unflipped_matrix)[1][1]

4×4 transpose(::Matrix{Int64}) with eltype Int64:
  0   1  -1  0
 -1   0   1  0
  0   0   1  0
 -1  -1   1  1

In [26]:
det(g)

1.0

In [27]:
transpose(g)*flipped_matrix

4×7 Matrix{Int64}:
 0   0  -1   0   1   0   1
 0   1   0   1  -1  -1   0
 1   0   1   1   0   0   0
 0  -1   0  -1   0   0  -1

## Compute lower dimensional cells

In [29]:
cell_orbits_in_dimension_5 = Matrix{Int64}[]
for higher_cell in cells_SL4[6]
    for j in 1:size(higher_cell)[2]
        boundary = higher_cell[begin:end,begin:end .!=j] # exclude the j-th column
        if cell_in_interior(boundary)
            # only go on if it lies in the interior of the symmetric space
            if !orbit_in_list(boundary,cell_orbits_in_dimension_5)
                push!(cell_orbits_in_dimension_5,boundary)
            end
        end
    end
end
cells_SL4[5] = cell_orbits_in_dimension_5

4-element Vector{Matrix{Int64}}:
 [0 0 … 1 1; 1 1 … 0 0; 0 0 … -1 0; -1 0 … 0 -1]
 [0 0 … 1 1; 0 1 … 0 0; 1 0 … -1 0; 0 0 … 0 -1]
 [0 0 … 1 1; 0 1 … 0 0; 1 0 … -1 0; 0 -1 … 0 -1]
 [0 0 … 1 1; 0 1 … 0 0; 1 0 … 0 0; 0 -1 … -1 0]

In [30]:
cell_orbits_in_dimension_4 = Matrix{Int64}[]
for higher_cell in cells_SL4[5]
    for j in 1:size(higher_cell)[2]
        boundary = higher_cell[begin:end,begin:end .!=j] # exclude the j-th column
        if cell_in_interior(boundary)
            # only go on if it lies in the interior of the symmetric space
            if !orbit_in_list(boundary,cell_orbits_in_dimension_4)
                push!(cell_orbits_in_dimension_4,boundary)
            end
        end
    end
end
cells_SL4[4] = cell_orbits_in_dimension_4

3-element Vector{Matrix{Int64}}:
 [0 0 … 1 1; 1 1 … 0 0; 0 1 … -1 0; 0 -1 … 0 -1]
 [0 0 … 1 1; 1 1 … 0 0; 0 1 … -1 0; -1 -1 … 0 -1]
 [0 0 … 1 1; 1 1 … -1 0; 0 0 … 0 -1; -1 0 … 0 0]

In [31]:
cell_orbits_in_dimension_3 = Matrix{Int64}[]
for higher_cell in cells_SL4[4]
    for j in 1:size(higher_cell)[2]
        boundary = higher_cell[begin:end,begin:end .!=j] # exclude the j-th column
        if cell_in_interior(boundary)
            # only go on if it lies in the interior of the symmetric space
            if !orbit_in_list(boundary,cell_orbits_in_dimension_3)
                push!(cell_orbits_in_dimension_3,boundary)
            end
        end
    end
end
cells_SL4[3] = cell_orbits_in_dimension_3

1-element Vector{Matrix{Int64}}:
 [0 1 1 1; 1 -1 0 0; 1 0 -1 0; -1 0 0 -1]

Also here, the SL_4 orbits seem to be the same as the GL_4 orbits

In [32]:
# export the cells to redo computations with the SL_5 code
serialize(joinpath(@__DIR__, "./sl4_cells.sjl"), cells_SL4)

# Stabilisers

In [33]:
stabilisers = Dict()
for (dimension, cell_list) in cells_SL4
    stabilisers_this_dimension = []
    for cell in cell_list
        push!(stabilisers_this_dimension,stabiliser_coset_with_orientation(cell,cell))
    end
    stabilisers[dimension] = stabilisers_this_dimension
end

In [32]:
for (dim,cells) in stabilisers
    println("\n Dimension $dim\n")
    for cell in cells
        size = length(cell)
        println("Stabiliser size: $size")
        preserves_orientation = true
        for element in cell
            if element[2] == -1
                preserves_orientation = false
            end
        end
        if preserves_orientation
            println("The stabiliser preserves the orientation.")
        else
            println("The stabiliser can revert the orientation.")
        end
    end
end


 Dimension 5

Stabiliser size: 12
The stabiliser preserves the orientation.
Stabiliser size: 48
The stabiliser can revert the orientation.
Stabiliser size: 16
The stabiliser can revert the orientation.
Stabiliser size: 144
The stabiliser can revert the orientation.

 Dimension 4

Stabiliser size: 48
The stabiliser can revert the orientation.
Stabiliser size: 48
The stabiliser can revert the orientation.
Stabiliser size: 120
The stabiliser preserves the orientation.

 Dimension 6

Stabiliser size: 8
The stabiliser can revert the orientation.
Stabiliser size: 8
The stabiliser can revert the orientation.
Stabiliser size: 48
The stabiliser preserves the orientation.
Stabiliser size: 48
The stabiliser can revert the orientation.

 Dimension 7

Stabiliser size: 8
The stabiliser can revert the orientation.
Stabiliser size: 8
The stabiliser can revert the orientation.

 Dimension 8

Stabiliser size: 12
The stabiliser preserves the orientation.
Stabiliser size: 72
The stabiliser can revert the

# Boundaries

In [34]:
boundaries_SL4 = Dict()
for (dimension, cell_list) in cells_SL4
    boundaries_this_dimension = []
    for cell in cell_list
        push!(boundaries_this_dimension,boundaries_in_group_ring_with_orientation(cell,cells_SL4))
    end
    boundaries_SL4[dimension] = boundaries_this_dimension
end

In [35]:
boundaries_SL4

Dict{Any, Any} with 6 entries:
  5 => Any[Any[Dict{Any, Any}("sign"=>1, "orbit_coset_with_orientation"=>Any[([…
  4 => Any[Any[Dict{Any, Any}("sign"=>1, "orbit_coset_with_orientation"=>Any[([…
  6 => Any[Any[Dict{Any, Any}("sign"=>1, "orbit_coset_with_orientation"=>Any[([…
  7 => Any[Any[Dict{Any, Any}("sign"=>1, "orbit_coset_with_orientation"=>Any[([…
  8 => Any[Any[Dict{Any, Any}("sign"=>1, "orbit_coset_with_orientation"=>Any[([…
  3 => Any[Any[]]

In [36]:
sl4_data = Dict()
sl4_data["boundaries"] = boundaries_SL4
sl4_data["stabilisers"] = stabilisers;

In [37]:
serialize(joinpath(@__DIR__, "../../scripts/differentials_computation/precomputed_boundaries/sl4_bound_stab_old.sjl"), sl4_data)

In [38]:
boundaries_SL4[4][2]

3-element Vector{Any}:
 Dict{Any, Any}("sign" => 1, "orbit_coset_with_orientation" => Any[([1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1], 1), ([-1 0 -2 2; 0 -1 -2 2; -2 0 -1 2; -2 0 -2 3], 1), ([-1 -2 0 2; -2 -1 0 2; 0 -2 -1 2; -2 -2 0 3], 1), ([3 -2 -2 0; 2 -1 -2 0; 2 -2 -1 0; 4 -2 -2 -1], 1), ([-3 2 2 0; -2 1 2 0; -2 2 1 0; -4 2 2 1], 1), ([1 2 0 -2; 2 1 0 -2; 0 2 1 -2; 2 2 0 -3], 1), ([1 0 2 -2; 0 1 2 -2; 2 0 1 -2; 2 0 2 -3], 1), ([-1 0 0 0; 0 -1 0 0; 0 0 -1 0; 0 0 0 -1], 1), ([1 -2 0 0; 0 -1 0 0; 0 -2 0 1; 0 -2 1 0], -1), ([-1 2 2 -2; 0 1 2 -2; -2 2 2 -1; -2 2 3 -2], -1)  …  ([1 -2 -2 2; 0 -2 -1 2; 1 -1 -1 1; 2 -3 -2 2], -1), ([1 -2 0 0; 2 -2 -1 0; 1 -1 1 -1; 2 -3 0 0], -1), ([3 -2 -2 0; 2 -2 -1 0; 2 -1 -2 0; 3 -3 -3 1], 1), ([-1 -2 0 2; 0 -2 -1 2; -2 -1 0 2; -1 -3 -1 3], 1), ([-1 0 -2 2; -2 0 -1 2; 0 -1 -2 2; -1 -1 -3 3], 1), ([1 0 0 0; 0 0 1 0; 0 1 0 0; 1 1 1 -1], 1), ([-1 0 0 0; 0 0 -1 0; 0 -1 0 0; -1 -1 -1 1], 1), ([1 0 2 -2; 2 0 1 -2; 0 1 2 -2; 1 1 3 -3], 1), ([1 2 0 -2; 0 2 1 -2; 2 1 

## Some sanity checks

In [38]:
for cell in  boundaries_SL4[4]
    println("A cell boundary looks like this:")
    for boundary in cell
        print("\n cell orbit: ")
        print(boundary["orbit_standard_cell"])
        print("\n Coset size: ")
        print(length(boundary["orbit_coset_with_orientation"]))
    end
    println()
end

A cell boundary looks like this:

 cell orbit: 1
 Coset size: 192
 cell orbit: 1
 Coset size: 192
 cell orbit: 1
 Coset size: 192
 cell orbit: 1
 Coset size: 192
A cell boundary looks like this:

 cell orbit: 1
 Coset size: 192
 cell orbit: 1
 Coset size: 192
 cell orbit: 1
 Coset size: 192
A cell boundary looks like this:

 cell orbit: 1
 Coset size: 192
 cell orbit: 1
 Coset size: 192
 cell orbit: 1
 Coset size: 192
 cell orbit: 1
 Coset size: 192
 cell orbit: 1
 Coset size: 192


In [39]:
for cell in  boundaries_SL4[5]
    println("A cell boundary looks like this:")
    for boundary in cell
        print("\n cell orbit: ")
        print(boundary["orbit_standard_cell"])
        print("\n Coset size: ")
        print(length(boundary["orbit_coset_with_orientation"]))
    end
    println()
end

A cell boundary looks like this:

 cell orbit: 1
 Coset size: 48
 cell orbit: 2
 Coset size: 48
 cell orbit: 2
 Coset size: 48
 cell orbit: 1
 Coset size: 48
 cell orbit: 2
 Coset size: 48
 cell orbit: 3
 Coset size: 120
A cell boundary looks like this:

 cell orbit: 1
 Coset size: 48
 cell orbit: 1
 Coset size: 48
 cell orbit: 1
 Coset size: 48
 cell orbit: 1
 Coset size: 48
 cell orbit: 1
 Coset size: 48
 cell orbit: 1
 Coset size: 48
A cell boundary looks like this:

 cell orbit: 2
 Coset size: 48
 cell orbit: 1
 Coset size: 48
 cell orbit: 2
 Coset size: 48
 cell orbit: 2
 Coset size: 48
 cell orbit: 2
 Coset size: 48
A cell boundary looks like this:

 cell orbit: 2
 Coset size: 48
 cell orbit: 2
 Coset size: 48
 cell orbit: 2
 Coset size: 48
 cell orbit: 2
 Coset size: 48
 cell orbit: 2
 Coset size: 48
 cell orbit: 2
 Coset size: 48


In [40]:
for cell in  boundaries_SL4[6]
    println("A cell boundary looks like this:")
    for boundary in cell
        print("\n cell orbit: ")
        print(boundary["orbit_standard_cell"])
        print("\n Coset size: ")
        print(length(boundary["orbit_coset_with_orientation"]))
    end
    println()
end

A cell boundary looks like this:

 cell orbit: 1
 Coset size: 12
 cell orbit: 2
 Coset size: 48
 cell orbit: 3
 Coset size: 16
 cell orbit: 1
 Coset size: 12
 cell orbit: 1
 Coset size: 12
 cell orbit: 3
 Coset size: 16
 cell orbit: 1
 Coset size: 12
A cell boundary looks like this:

 cell orbit: 3
 Coset size: 16
 cell orbit: 3
 Coset size: 16
 cell orbit: 4
 Coset size: 144
 cell orbit: 1
 Coset size: 12
 cell orbit: 3
 Coset size: 16
 cell orbit: 3
 Coset size: 16
 cell orbit: 1
 Coset size: 12
A cell boundary looks like this:

 cell orbit: 3
 Coset size: 16
 cell orbit: 3
 Coset size: 16
 cell orbit: 3
 Coset size: 16
 cell orbit: 3
 Coset size: 16
 cell orbit: 3
 Coset size: 16
 cell orbit: 3
 Coset size: 16
A cell boundary looks like this:

 cell orbit: 3
 Coset size: 16
 cell orbit: 3
 Coset size: 16
 cell orbit: 3
 Coset size: 16
 cell orbit: 3
 Coset size: 16
 cell orbit: 3
 Coset size: 16
 cell orbit: 2
 Coset size: 48
 cell orbit: 3
 Coset size: 16


In [42]:
for cell in  boundaries_SL4[7]
    println("A cell boundary looks like this:")
    for boundary in cell
        print("\n cell orbit: ")
        print(boundary["orbit_standard_cell"])
        print("\n Coset size: ")
        print(length(boundary["orbit_coset_with_orientation"]))
    end
    println()
end

A cell boundary looks like this:

 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 2
 Coset size: 8
 cell orbit: 2
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 2
 Coset size: 8
 cell orbit: 2
 Coset size: 8
 cell orbit: 1
 Coset size: 8
A cell boundary looks like this:

 cell orbit: 3
 Coset size: 48
 cell orbit: 2
 Coset size: 8
 cell orbit: 2
 Coset size: 8
 cell orbit: 2
 Coset size: 8
 cell orbit: 3
 Coset size: 48
 cell orbit: 4
 Coset size: 48
 cell orbit: 1
 Coset size: 8
 cell orbit: 2
 Coset size: 8


In [43]:
for cell in  boundaries_SL4[8]
    println("A cell boundary looks like this:")
    for boundary in cell
        print("\n cell orbit: ")
        print(boundary["orbit_standard_cell"])
        print("\n Coset size: ")
        print(length(boundary["orbit_coset_with_orientation"]))
    end
    println()
end

A cell boundary looks like this:

 cell orbit: 2
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 2
 Coset size: 8
 cell orbit: 2
 Coset size: 8
 cell orbit: 2
 Coset size: 8
 cell orbit: 2
 Coset size: 8
 cell orbit: 2
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
A cell boundary looks like this:

 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8


# SL_3 redone
To check both the new code and the old results. At least the sizes of stabilisers etc coincide.

In [41]:
e1 = [1,0,0]
e2 = [0,1,0]
e3 = [0,0,1]
e12 = [1,1,0]
e13 = [1,0,1]
e123 = [1,1,1]
m2 = [e1 e2 e3] # standard
m3_1 = [e1 e2 e3 e12] # 2-additive
m3_2 = [e1 e2 e3 e123] # 3-additive
m4 = [e1 e2 e3 e12 e13] # double-triple
m5 = [e1 e2 e3 e12 e13 e123] # Q
cells_SL3 = Dict()
cells_SL3[2] = [m2]
cells_SL3[3] = [m3_1,m3_2]
cells_SL3[4] = [m4]
cells_SL3[5] = [m5]

1-element Vector{Matrix{Int64}}:
 [1 0 … 1 1; 0 1 … 0 1; 0 0 … 1 1]

In [42]:
stabilisers = Dict()
for (dimension, cell_list) in cells_SL3
    stabilisers_this_dimension = []
    for cell in cell_list
        push!(stabilisers_this_dimension,stabiliser_coset_with_orientation(cell,cell))
    end
    stabilisers[dimension] = stabilisers_this_dimension
end

In [43]:
boundaries_SL3 = Dict()
for (dimension, cell_list) in cells_SL3
    boundaries_this_dimension = []
    for cell in cell_list
        push!(boundaries_this_dimension,boundaries_in_group_ring_with_orientation(cell,cells_SL3))
    end
    boundaries_SL3[dimension] = boundaries_this_dimension
end

In [44]:
boundaries_SL3

Dict{Any, Any} with 4 entries:
  5 => Any[Any[Dict{Any, Any}("sign"=>1, "orbit_coset_with_orientation"=>Any[([…
  4 => Any[Any[Dict{Any, Any}("sign"=>1, "orbit_coset_with_orientation"=>Any[([…
  2 => Any[Any[]]
  3 => Any[Any[Dict{Any, Any}("sign"=>1, "orbit_coset_with_orientation"=>Any[([…

In [45]:
sl3_data = Dict()
sl3_data["boundaries"] = boundaries_SL3
sl3_data["stabilisers"] = stabilisers;

In [46]:
serialize("../../scripts/differentials_computation/precomputed_boundaries/sl3_bound_stab_old.sjl", sl3_data)

In [50]:
for cell in  boundaries_SL3[3]
    println("A cell boundary looks like this:")
    for boundary in cell
        print("\n cell orbit: ")
        print(boundary["orbit_standard_cell"])
        print("\n Coset size: ")
        print(length(boundary["orbit_coset_with_orientation"]))
    end
    println()
end

A cell boundary looks like this:

 cell orbit: 1
 Coset size: 24
 cell orbit: 1
 Coset size: 24
 cell orbit: 1
 Coset size: 24
A cell boundary looks like this:

 cell orbit: 1
 Coset size: 24
 cell orbit: 1
 Coset size: 24
 cell orbit: 1
 Coset size: 24
 cell orbit: 1
 Coset size: 24


In [51]:
for cell in  boundaries_SL3[4]
    println("A cell boundary looks like this:")
    for boundary in cell
        print("\n cell orbit: ")
        print(boundary["orbit_standard_cell"])
        print("\n Coset size: ")
        print(length(boundary["orbit_coset_with_orientation"]))
    end
    println()
end

A cell boundary looks like this:

 cell orbit: 2
 Coset size: 24
 cell orbit: 1
 Coset size: 12
 cell orbit: 1
 Coset size: 12
 cell orbit: 1
 Coset size: 12
 cell orbit: 1
 Coset size: 12


In [52]:
for cell in  boundaries_SL3[5]
    println("A cell boundary looks like this:")
    for boundary in cell
        print("\n cell orbit: ")
        print(boundary["orbit_standard_cell"])
        print("\n Coset size: ")
        print(length(boundary["orbit_coset_with_orientation"]))
    end
    println()
end

A cell boundary looks like this:

 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
 cell orbit: 1
 Coset size: 8
